Proyecto jiji

In [85]:
!pip install kagglehub


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import regex as re

[nltk_data] Downloading package punkt to C:\Users\Ce
[nltk_data]     mar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [87]:
import os
import pandas as pd
import pickle
import math
import sys
import dbm
stemmer = SnowballStemmer('english')

In [88]:
with open("stopwords-en.txt", encoding="latin1") as file:
   stoplist = [line.rstrip().lower() for line in file]
stoplist += ['?', '-', '.', ':', ',', '!', ';']

def preprocesamiento(texto, stemming=True):
  words = []
  texto = str(texto)
  texto = texto.lower()
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # tokenizar
  words = nltk.word_tokenize(texto, language='spanish')
  # filtrar stopwords
  words = [word for word in words if word not in stoplist]
  # reducir palabras (stemming)
  if stemming:
      words = [stemmer.stem(word) for word in words]
  return words


In [89]:
def OrdenarPorBloques(dir_blocks, n_blocks):
    for i in range(n_blocks):
        file_path = os.path.join(dir_blocks, 'block_{}.pkl'.format(i))
        with open(file_path, 'rb') as f:
            tuplas_ordenadas = pickle.load(f)
        tuplas_ordenadas = sorted(list(tuplas_ordenadas.items()), key=lambda x: x[0])
        # tuplas_ordenadas = [par for par in tuplas_ordenadas if not par[0].isdigit()]
        tuplas_ordenadas = [(term[0], list(term[1].items())) for term in tuplas_ordenadas]
        with open(file_path, 'wb') as f:
            pickle.dump(tuplas_ordenadas, f)

def mergeSortAux(dir_bloques, l, r):
    if l == r:
        bloque = leer_bloque(dir_bloques, l)
        unicos = set()    
        for par in bloque:
            unicos.add(par[0])
        return list(unicos)
    
    if (l < r):
        mid = int(math.ceil((r + l)/2.0))
        unique_l = mergeSortAux(dir_bloques, l, mid - 1)
        unique_r = mergeSortAux(dir_bloques, mid, r)
        unicos = set()    
        for term in unique_l:
            unicos.add(term)
        for term in unique_r:
            unicos.add(term)
        unicos = list(unicos)
        merge_v2(dir_bloques, l, r, mid, len(unicos))
        return list(unicos)
        
    return []

def escribir_bloque(dir_bloques, block, idx_insert_block, buffer_limit = 2000):
    with open(os.path.join(dir_bloques, "block_{}_v2.pkl".format(idx_insert_block)), 'wb') as f:
        pickle.dump(block, f)    
def leer_bloque(dir_bloques, it):
    file_path = os.path.join(dir_bloques, f"block_{it}.pkl")
    with open(file_path, "rb") as f:
        buffer = pickle.load(f)
    return buffer
def merge_v2(dir_bloques, l, r, mid, num_terms):
    idx_insert_block = l
    new_block = []
    mezclar_n_bloques = r - l + 1
    unique_terms_per_block = int(math.ceil(num_terms/mezclar_n_bloques))
    unique_terms_current_block = 0

    it_l = l
    it_r = mid
    term_dic_l = leer_bloque(dir_bloques, it_l)
    term_dic_r = leer_bloque(dir_bloques, it_r)
    
    idx_term_l = 0
    idx_term_r = 0

    idx_doc_l = 0
    idx_doc_r = 0
    new_block = []
    while(it_l < mid and it_r < r + 1):
        while(idx_term_l < len(term_dic_l) and idx_term_r < len(term_dic_r)): # moverme entre palabras de dos bloques
            new_term = []
            if(term_dic_l[idx_term_l][0] < term_dic_r[idx_term_r][0]):
                new_term = term_dic_l[idx_term_l]
                idx_term_l += 1
            elif(term_dic_l[idx_term_l][0] > term_dic_r[idx_term_r][0]):
                new_term = term_dic_r[idx_term_r]
                idx_term_r += 1
            else:
                idx_doc_l = 0
                idx_doc_r = 0
                while(idx_doc_l < len(term_dic_l[idx_term_l][1]) and idx_doc_r < len(term_dic_r[idx_term_r][1])):
                    # print(f"Toma 2 terminos iguales con tf = {term_dic_l[idx_term_l][1]} y {term_dic_r[idx_term_r][1]}")
                    if term_dic_l[idx_term_l][1][idx_doc_l][0] > term_dic_r[idx_term_r][1][idx_doc_r][0]:
                        pushear_doc = term_dic_r[idx_term_r][1][idx_doc_r]
                        idx_doc_r += 1
                    elif term_dic_l[idx_term_l][1][idx_doc_l][0] < term_dic_r[idx_term_r][1][idx_doc_r][0]:
                        pushear_doc = term_dic_l[idx_term_l][1][idx_doc_l]
                        idx_doc_l += 1
                    else:
                        pushear_doc = (term_dic_l[idx_term_l][1][idx_doc_l][0], term_dic_l[idx_term_l][1][idx_doc_l][1] + term_dic_r[idx_term_r][1][idx_doc_r][1])
                        idx_doc_l += 1
                        idx_doc_r += 1
                    new_term.append(pushear_doc)
                while(idx_doc_l < len(term_dic_l[idx_term_l][1])):
                    pushear_doc = term_dic_l[idx_term_l][1][idx_doc_l]
                    idx_doc_l += 1
                    new_term.append(pushear_doc)
                while(idx_doc_r < len(term_dic_r[idx_term_r][1])):
                    pushear_doc = term_dic_r[idx_term_r][1][idx_doc_r]
                    idx_doc_r += 1
                    new_term.append(pushear_doc)
                new_term = (term_dic_l[idx_term_l][0], new_term)
                idx_term_r += 1
                idx_term_l += 1
            new_block.append(new_term)
            
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
        if(len(term_dic_l) == idx_term_l):
            if (it_l < mid - 1):
                it_l += 1
                term_dic_l = leer_bloque(dir_bloques, it_l)
                idx_term_l = 0
                idx_doc_l = 0
                continue
            else:
                break
        if(len(term_dic_r) == idx_term_r):
            if (it_r < r):
                it_r += 1
                term_dic_r = leer_bloque(dir_bloques, it_r)
                idx_term_r = 0
                idx_doc_r = 0
                continue
            else:
                break
        if(it_l == mid | it_r == r + 1):
            break
    while(it_l < mid):
        term_dic_l = leer_bloque(dir_bloques, it_l)
        while(idx_term_l < len(term_dic_l)):
            new_block.append(term_dic_l[idx_term_l])
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
            idx_term_l += 1
        idx_term_l = 0
        it_l += 1
    while(it_r < r + 1):
        term_dic_r = leer_bloque(dir_bloques, it_r)
        while(idx_term_r < len(term_dic_r)):
            new_block.append(term_dic_r[idx_term_r])
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
            idx_term_r += 1
        idx_term_r = 0
        it_r += 1

    while(idx_insert_block < r + 1):
        if len(new_block) > 0:
            escribir_bloque(dir_bloques, new_block, idx_insert_block)
        else:
            escribir_bloque(dir_bloques, [], idx_insert_block)
        new_block = []
        idx_insert_block += 1
    idx_insert_block = l
    for idx_archivo in range(l, r + 1):
        nuevo_nombre = os.path.join(dir_bloques, "block_{}.pkl".format(idx_archivo))
        if os.path.exists(nuevo_nombre):
            os.remove(nuevo_nombre)
        os.rename(os.path.join(dir_bloques, "block_{}_v2.pkl".format(idx_archivo)), nuevo_nombre)
def mergeSort(dir_bloques):
    bloques_files_dir = os.listdir(os.path.join('./',dir_bloques))
    # print(bloques_files_dir)
    n = len(bloques_files_dir)
    # n = int(math.exp2(math.floor(math.log2(n)) + 1))
    mergeSortAux(dir_bloques, 0, n - 1)
'''
Recibe la direccion del folder con los diccionarios del spimi,
modifica los archivos y los sobreescribe para crear el índice
invertido con un índice global
'''
def InvertirListasDiccionarios(dir_bloques):
    n_bloques = len(os.listdir(os.path.join('./',dir_bloques)))
    for i in range(n_bloques):
        bloque_path = os.path.join(dir_bloques,"block_{}.pkl".format(i))
        with open(bloque_path, 'rb') as f:
            bloque = pickle.load(f)
        bloque_dict = {}
        if len(bloque) != 0:
            for term, poosting_list in bloque:
                poosting_dict = {}
                for doc, tf in poosting_list:
                    poosting_dict[doc] = tf
                bloque_dict[term] = poosting_dict
        with open(bloque_path, 'wb') as f:
            pickle.dump(bloque_dict, f)

def getNumberWithAtributo(dataset_head, atributo):
    if atributo in dataset_head.columns:
        return dataset_head.columns.get_loc(atributo)  # Obtiene la posición del atributo
    return -1  # Retorna -1 si no existe

def CrearIndiceGlobal(dir_bloques, dir_global_index):
    path_global_index = os.path.join('./', dir_global_index)
    if not (os.path.exists(path_global_index) and os.path.isdir(path_global_index)):
        os.mkdir(path_global_index)
    path_global_index_file = os.path.join(path_global_index, "btree_index")
    with dbm.open(path_global_index_file, "c") as db:
        path_bloques = os.listdir(os.path.join('./',dir_bloques))
        for path in path_bloques:
            path_bloque = os.path.join(dir_bloques, path)
            with open(path_bloque, 'rb') as f:
                bloque = pickle.load(f)
            for term, poosting in bloque.items():
                db[term] = path
def PreComputarNormas(dir_bloques, dir_normas, n_docs, n_blocks, disk_buffer_size):
    
    norms = {}
    norm_path = os.path.join('./', dir_normas)
    if not (os.path.exists(norm_path) and os.path.isdir(norm_path)):
        os.mkdir(norm_path)
    
    for i in range(n_blocks):
        bloque_i_path = os.path.join(dir_bloques, "block_{}.pkl".format(i))
        with open(bloque_i_path, 'rb') as f:
            bloque_i = pickle.load(f)
        for term, poosting in bloque_i.items():
            df = len(poosting)
            idf = math.log10(n_docs / df)
            for doc, tf in poosting.items():
                itf = math.log10(1 + tf)
                if doc not in norms.keys():
                    norms[doc] = 0
                norms[doc] += (itf*idf)**2
    for doc, peso in norms.items():
        norms[doc] = math.sqrt(peso)
    buffer = {}
    index_buffer = 0
    for doc, peso in sorted(norms.items(), key=lambda x: x[0]):
        buffer[doc] = peso
        if sys.getsizeof(buffer) >= disk_buffer_size:
            with open(os.path.join(dir_normas,"norma_block_{}.pkl".format(index_buffer)), 'wb') as f:
                pickle.dump(buffer, f)
                buffer = {}
                index_buffer += 1
    if buffer:
        with open(os.path.join(dir_normas,"norma_block_{}.pkl".format(index_buffer)), 'wb') as f:
            pickle.dump(buffer, f)
            buffer = {}
            index_buffer += 1
    return index_buffer
def getResultados(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            if doc in chunk.index:
                res.append((chunk.iloc[doc], score))
    return pd.DataFrame(res, columns=['doc_id', 'score'])    
def get_dfText(path, col, row, disk_limit):
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        if row in chunk.index:
            return chunk.iloc[row,col]
    return ""

def get_dfTex_Cols(path, row, columnas): #We don´t use it is only ofr simple an very fast testing
    dataset =  pd.read_csv(path)
    res = [dataset.iloc[row, i] for i in columnas]
    return ' '.join(res)  

In [90]:
import os
import pickle
import sys
import math
import heapq

class SPIMI:
    def __init__(self, index_dataset="path", position = 3, col_text = "lyrics", columnas = ["track_name","track_artist","lyrics", "track_album_name"], index_dir="index_blocks", idf_dir="normas", global_index_dir ="global_index"):
        self.index_dir = index_dir  
        self.idf_dir = idf_dir
        self.global_index_dir = global_index_dir
        self.path_ = index_dataset 
        self.block_counter = 0      
        self.disk_limit = 4000  
        self.position = position
        self.col_text = col_text
        self.norm_bloques = 0
        self.n_docs = 0
        #Define columns to crear SPIMI
        self.columnas = columnas 

        if not os.path.exists(self.index_dir):
            os.makedirs(self.index_dir)
    def cargar_indice(self, dir_indice_invertido):
        with open(dir_indice_invertido, 'rb') as f:
            dict_info = pickle.load(f)

        self.index_dir = dict_info['index_dir']
        self.idf_dir = dict_info['idf_dir']
        self.global_index_dir = dict_info['global_index_dir']
        self.path_ = dict_info['path_']
        self.block_counter = dict_info['block_counter']
        self.disk_limit = dict_info['disk_limit']
        self.position = dict_info['position']
        self.col_text = dict_info['col_text']
        self.norm_bloques = dict_info['norm_bloques']
        self.n_docs = dict_info['n_docs']
    def guardar_indice(self):
        info_indice = {
            "index_dir": self.index_dir,
            "idf_dir": self.idf_dir,
            "global_index_dir": self.global_index_dir,
            "path_": self.path_,
            "block_counter": self.block_counter,
            "disk_limit": self.disk_limit,
            "position": self.position,
            "col_text": self.col_text,
            "norm_bloques": self.norm_bloques,
            "n_docs": self.n_docs
        }
        with open("indice_invertido.pkl", 'wb') as f:
            pickle.dump(info_indice, f)
    def spimi_invert(self):
        dictionary = {}
        doc_ids = set()
        stop = False

        #Get columna indeces: 
        with open(self.path_, mode='r', encoding='utf-8') as file:
            primera_linea = file.readline().strip()
        #Get number columns
        columnas = primera_linea.split(',')
        columnas_numbers  = [i for i in range(len(columnas)) if columnas[i] in self.columnas]
        for chunk in pd.read_csv(self.path_, chunksize=self.disk_limit):
            for doc_id_, row in chunk.iterrows():
                preFila = [row.iloc[i] for i in columnas_numbers]
                texto = ' '.join(preFila)
                words = preprocesamiento(texto)
                for text in words:
                    doc_id = doc_id_
                    token = text
                    doc_ids.add(doc_id)
                    if token not in dictionary:
                        dictionary[token] = {}  

                    if doc_id not in dictionary[token]:
                        dictionary[token][doc_id] = 1  
                    else:
                        dictionary[token][doc_id] += 1  

                    dictionary_size = sys.getsizeof(dictionary)
                    if dictionary_size >= self.disk_limit:
                        self.write_block_to_disk(dictionary, level=0)
                        dictionary.clear()
        if dictionary:
            self.write_block_to_disk(dictionary, level=0)
        self.n_docs = len(doc_ids)
        OrdenarPorBloques(dir_blocks=self.index_dir, n_blocks=self.block_counter)
        mergeSort(self.index_dir)
        InvertirListasDiccionarios(self.index_dir)
        CrearIndiceGlobal(self.index_dir, self.global_index_dir)
        self.norm_bloques = PreComputarNormas(self.index_dir, self.idf_dir, self.n_docs, self.block_counter, self.disk_limit)
        self.guardar_indice()
        
    def load_index(self):
        self.index = {}
        files = sorted(os.listdir(self.index_dir))
        for file in files:
            if file.endswith(".pkl"):
                block = self.load_block(os.path.join(self.index_dir, file))
                for term, postings in block.items():
                    if term not in self.index:
                        self.index[term] = []
                    for doc_id, tf in postings.items():
                        
                        self.index[term].append((doc_id, math.log10(1+ tf)))
 
    def load_block(self, filepath):
        """ Cargar un bloque del índice desde un archivo. """
        with open(filepath, "rb") as f:
            return pickle.load(f)
        


    def write_block_to_disk(self, dictionary, level):
        sorted_terms = dict(sorted(dictionary.items())) 
        # block_{it}
        file_path = os.path.join(self.index_dir, f"block_{self.block_counter}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(sorted_terms, f)
        
        self.block_counter += 1

    def load_block(self, filepath):
        with open(filepath, "rb") as f:
            return pickle.load(f)
    def get_term(self, term, path_bloque):
        complete_list = {}
        with open(os.path.join(self.index_dir, path_bloque), 'rb') as f:
            complete_list = pickle.load(f)
        if term not in complete_list.keys():
            print("Alerta ", term)
        return complete_list[term]
    
    def retrieval(self, query, k):
        N = self.n_docs  
        scores = [0] * N  
        tf_query = {}  
        terms = preprocesamiento(query)  

        for term in terms:
            if term in tf_query:
                tf_query[term] += 1
            else:
                tf_query[term] = 1

        tfidf_query = {}
        with dbm.open(os.path.join(self.global_index_dir, "btree_index"), "c") as db:
            for term, tf in tf_query.items():
                if term in db:
                    poosting_list = self.get_term(term, db[term].decode("utf-8"))
                    df_term = len(poosting_list)
                    tfidf_query[term] = math.log10(1 + tf) * math.log10(self.n_docs/df_term)
        norm_query = math.sqrt(sum(w_tq**2 for w_tq in tfidf_query.values()))  # Normalización del query

        # Aplicar similitud de coseno: Calculamos el puntaje para cada documento
        with dbm.open(os.path.join(self.global_index_dir, "btree_index"), "c") as db:
            for term, w_tq in tfidf_query.items():
                if term in db:
                    poosting_list = self.get_term(term, db[term].decode("utf-8"))
                    df_term = len(poosting_list)
                    idf_term = math.log10(self.n_docs/df_term)
                    for doc, tf_td in poosting_list.items():
                        w_td = math.log10(1 + tf_td) * idf_term
                        # print("self.index", term, "es", self.index[term])
                        scores[doc] += w_td * w_tq #Producto punto

        # Normalizar las puntuaciones de los documentos
        # idx_doc = 0
        for i in range(self.norm_bloques):
            with open(os.path.join(self.idf_dir, "norma_block_{}.pkl".format(i)), 'rb') as f:
                normas = pickle.load(f)
            for doc, peso in normas.items():
                scores[doc] /= (peso * norm_query)
            

        # Ordenar las puntuaciones en orden descendente
        result = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)

        # Devolver los k documentos más relevantes (top-k)
        return result[:k]

In [91]:
# --------Set Path----------
import kagglehub
path = kagglehub.dataset_download("imuhammad/audio-features-and-lyrics-of-spotify-songs")

In [92]:
# ----------Set dataset--------

lista_ = os.listdir(path)
songs = os.path.join(path, lista_[0])
print("Path de songs: ",songs)


Path de songs:  C:\Users\Ce mar\.cache\kagglehub\datasets\imuhammad\audio-features-and-lyrics-of-spotify-songs\versions\1\spotify_songs.csv


In [ ]:
# --------Set Path----------
import kagglehub
path = kagglehub.dataset_download("imuhammad/audio-features-and-lyrics-of-spotify-songs")

# ----------Set dataset--------

lista_ = os.listdir(path)
songs = os.path.join(path, lista_[0])
print("Path de songs: ",songs)
# dataset = pd.read_csv(songs)
# dataset = dataset.head(20)


# ----------Set Atributos--------

columna = "lyrics"

# ---------Creation---------

# s = SPIMI(songs)  #Put songs path for real
s = SPIMI("spotify_20.csv") #Put this for experimentation
# s.spimi_invert()
s.cargar_indice(dir_indice_invertido="indice_invertido.pkl")
# ---------End Creation---------


#-----------Start of query------------

query = get_dfText(songs, 3, 0, s.disk_limit)
# query = get_dfTex_Cols(songs, 2, [1,2,3,6])
top_k = 5

result = s.retrieval(query, top_k)
getResultados(result, s.path_, s.disk_limit) # La respuesta es un array pd de [fila del dataframe, score]
#-----------End of query------------




Path de songs:  C:\Users\Ce mar\.cache\kagglehub\datasets\imuhammad\audio-features-and-lyrics-of-spotify-songs\versions\1\spotify_songs.csv


,doc_id,score
0,track_id ...,1.000000
1,track_id ...,0.063394
2,track_id ...,0.051552
3,track_id ...,0.048244
4,track_id ...,0.047911
